## 데이터 분리

### **개념**
- 전체 데이터를 **학습(Training)데이터** 와 **테스트(Test)데이터** 로 나누어 모델을 평가  
- 모델이 학습한 데이터와는 다른 새로운 데이터에서도 잘 동작하는지 확인하기 위함 (**일반화 성능 검증**)  

### **데이터 분리 목적**
1. **학습 데이터 (Training Set)**  
   - 모델의 **파라미터(가중치 등)** 를 학습하는 데 사용  
   - 입력 → 출력 관계를 학습  

2. **테스트 데이터 (Test Set)**  
   - 학습된 모델의 **최종 성능 평가** 를 위해 사용  
   - 학습 과정에서는 절대 사용하지 않음 → 새로운 데이터로 모델의 일반화 능력을 측정  

### **분리 비율**
- 데이터 크기에 따라 다르지만 보통 **학습:테스트 = 70:30 또는 80:20**  

### **검증 데이터(Validation Set)**
- 학습 데이터 일부를 **검증(Validation) 데이터** 로 나누어 모델 튜닝에 활용  
- 역할:
  - 하이퍼파라미터 조정  
    - 하이퍼파라미터(Hyperparameter): 학습에서 최적화되지 않고 사람이 직접 설정하는 모델의 설정값  
  - 모델 선택 비교  
  - 과적합 방지  

<p align="center"> <img src="https://raw.githubusercontent.com/isjo-dit/Lecture/4ff6a8a051a48c646ede2398d9ba1789e051daee/image/data_split.png" align='center' width=600> </p>

### **교차 검증(Cross Validation)**
- 데이터를 여러 조각(folds)으로 나누어 번갈아 학습/검증에 사용  
  - 5-folds 교차 검증: 데이터를 5 조각으로 분리하여 4 조각은 학습, 1 조각은 검증에 사용  
    → 모든 조각이 검증에 사용되도록 반복  
    
- 교차 검증의 장점  
  - 모든 데이터가 검증에 사용되어 더 신뢰할 수 있는 성능 평가  
  - 데이터가 부족할 때 특히 유용  
  - 모델의 안정성을 확인할 수 있음  

<p align="center"> <img src="https://raw.githubusercontent.com/isjo-dit/Lecture/4ff6a8a051a48c646ede2398d9ba1789e051daee/image/cv.png" align='center' width=600> </p>


<br>

---

## 모델 최적화


### Bagging (배깅)

<p align="center">
  <img src="https://aiml.com/wp-content/uploads/2023/03/Bagging-2-1536x971.png" align='center' width=900>
  <br>
  <sub><a href="https://aiml.com/what-is-bagging/">Bagging - Source: AIML.com </a></sub>
</p>

개념 : Bagging(Bootstrap Aggregating)은 여러 모델을 학습시켜 결과를 평균내는 앙상블 기법입니다.

**핵심 아이디어:**
- 동일한 데이터에서 **복원추출**로 여러 샘플 생성  
- 각 샘플로 **독립적인 모델** 학습
- 예측 시 모든 모델의 **결과를 평균**(회귀) 또는 **다수결**(분류)
- **과적합 감소**, **모델 안정성 향상**, **분산 감소** 효과

**작동 원리:**
1. 원본 데이터(N개)에서 복원추출로 N개 샘플 생성 (Bootstrap)
2. 각 샘플로 개별 모델 훈련 (예: 100개 의사결정트리)
3. 새로운 데이터 예측 시 모든 모델의 결과 종합
4. 개별 모델의 오차가 상쇄되어 전체 성능 향상

**Bootstrap Sampling**

In [ ]:
import numpy as np
from sklearn.utils import resample

# 원본 데이터
data = np.array([1, 2, 3, 4, 5])

# Bootstrap 샘플링 (복원추출)
bootstrap_sample = resample(data, n_samples=len(data), random_state=42)
print(f"Bootstrap 샘플: {bootstrap_sample}")
# 출력: [4 3 4 1 5] (중복 허용)

### Random Forest (Bagging + 랜덤 특성 선택)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 생성
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Random Forest (Bagging 기반)
rf = RandomForestClassifier(
    n_estimators=100,  # 트리 개수
    max_features='sqrt',  # 랜덤 특성 선택
    random_state=42
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(f"Random Forest 정확도: {accuracy_score(y_test, y_pred):.3f}")

### Bagging Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# 기본 모델로 Decision Tree 사용
base_model = DecisionTreeClassifier(random_state=42)

# Bagging 적용
bagging = BaggingClassifier(
    estimator=base_model,
    n_estimators=50,  # 모델 개수
    random_state=42
)

bagging.fit(X_train, y_train)
y_pred_bagging = bagging.predict(X_test)
print(f"Bagging 정확도: {accuracy_score(y_test, y_pred_bagging):.3f}")

# 단일 모델과 비교
single_tree = DecisionTreeClassifier(random_state=42)
single_tree.fit(X_train, y_train)
y_pred_single = single_tree.predict(X_test)
print(f"단일 트리 정확도: {accuracy_score(y_test, y_pred_single):.3f}")

### Bagging의 장점

In [ ]:
# 분산 줄이기 효과 확인
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 여러 번 실험하여 성능 분산 비교
single_scores = []
bagging_scores = []

for i in range(20):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)

    # 단일 모델
    single = DecisionTreeClassifier(random_state=42)
    single.fit(X_train, y_train)
    single_scores.append(accuracy_score(y_test, single.predict(X_test)))

    # Bagging 모델
    bagging = BaggingClassifier(DecisionTreeClassifier(random_state=42), n_estimators=10, random_state=42)
    bagging.fit(X_train, y_train)
    bagging_scores.append(accuracy_score(y_test, bagging.predict(X_test)))

print(f"단일 모델 평균: {np.mean(single_scores):.3f}, 표준편차: {np.std(single_scores):.3f}")
print(f"Bagging 평균: {np.mean(bagging_scores):.3f}, 표준편차: {np.std(bagging_scores):.3f}")

## Boosting
머신러닝 앙상블 기법 중 하나로 sequential한 약한 학습기(weak learner)들을 여러 개 결합하여 예측 혹은 분류 성능을 높이는 알고리즘

<p align="center">
  <img src="https://www.ibm.com/content/dam/connectedassets-adobe-cms/worldwide-content/creative-assets/s-migr/ul/g/fc/4f/ensemble-learning-boosting.component.xl.ts=1744838363574.png/content/adobe-cms/us/en/think/topics/gradient-boosting/jcr:content/root/table_of_contents/body-article-8/image_180162990" align='center' width=900>
  <br>
  <sub><a href="https://www.ibm.com/think/topics/gradient-boosting">Gradient Boosting - Source: IBM </a></sub>
</p>



XGBoost

- n_estimators: 트리 개수 (많을수록 복잡, 과적합 위험).

- learning_rate (eta): 학습률. 작게 설정하면 성능 안정적이지만 더 많은 트리 필요.

- max_depth: 트리 깊이. 과적합 제어.

- subsample: 학습 샘플의 일부만 사용 (0~1). 과적합 방지.

- colsample_bytree: 피처 일부만 사용.

- reg_alpha, reg_lambda: L1/L2 정규화 강도.

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# X, y: 입력 데이터와 타깃
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(
    objective="reg:squarederror",
    booster="gbtree",
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse:.3f}")


---

## GridSearchCV (하이퍼파라미터 튜닝)

### 개념
GridSearchCV는 다양한 하이퍼파라미터 조합을 자동으로 시도하여 최적의 모델을 찾는 기법입니다.

**핵심 아이디어:**
- **격자(Grid) 탐색**: 지정한 매개변수들의 모든 조합을 체계적으로 시도
- **교차검증(CV)**: 각 조합마다 K-fold 교차검증으로 성능 평가  
- **자동 최적화**: 가장 좋은 성능을 보인 매개변수 조합 자동 선택
- **과적합 방지**: 교차검증으로 일반화 성능 측정
- **시간 vs 성능**: 모든 조합을 시도하므로 시간이 오래 걸림

**작동 원리:**
1. 하이퍼파라미터 조합 리스트 생성 (예: C=[0.1,1,10] × kernel=[linear,rbf])
2. 각 조합에 대해 K-fold 교차검증 수행
3. 평균 검증 성능이 가장 높은 조합 선택
4. 선택된 조합으로 전체 훈련 데이터에 최종 모델 훈련


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# 매개변수 그리드 정의
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# 기본 모델
svm = SVC(random_state=42)

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    cv=5,  # 5-fold 교차검증
    scoring='accuracy',
    n_jobs=-1  # 모든 CPU 사용
)

# 최적 매개변수 찾기
grid_search.fit(X_train, y_train)

print(f"최적 매개변수: {grid_search.best_params_}")
print(f"최적 점수: {grid_search.best_score_:.3f}")

# 테스트 데이터로 평가
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print(f"테스트 정확도: {test_score:.3f}")

### Random Forest 튜닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest 매개변수 그리드
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    rf_param_grid,
    cv=3,  # 시간 단축을 위해 3-fold
    scoring='accuracy',
    n_jobs=-1
)

rf_grid.fit(X_train, y_train)
print(f"RF 최적 매개변수: {rf_grid.best_params_}")
print(f"RF 최적 점수: {rf_grid.best_score_:.3f}")

### RandomizedSearchCV (시간 단축)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# 더 넓은 매개변수 범위
rf_random_param = {
    'n_estimators': randint(50, 300),
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

# RandomizedSearchCV (시간 효율적)
rf_random = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    rf_random_param,
    n_iter=20,  # 20번만 시도
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

rf_random.fit(X_train, y_train)
print(f"Random Search 최적 매개변수: {rf_random.best_params_}")
print(f"Random Search 최적 점수: {rf_random.best_score_:.3f}")

### 파이프라인과 함께 사용  ****

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 전처리 + 모델 파이프라인
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 파이프라인 매개변수 (단계명__매개변수명)
pipeline_params = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [3, 5, None],
    'scaler__with_mean': [True, False]
}

pipeline_grid = GridSearchCV(
    pipeline,
    pipeline_params,
    cv=3,
    scoring='accuracy'
)

pipeline_grid.fit(X_train, y_train)
print(f"파이프라인 최적 매개변수: {pipeline_grid.best_params_}")

---

## 파이프라인

데이터 전처리 과정과 모델 학습 과정을 하나로 묶어 관리하는 도구
- 코드 간결화: 여러 단계를 한 줄로 묶어서 관리 가능
- 데이터 누수 방지: 학습 데이터에서만 훈련, 검증 데이터에서는 훈련 X
- 재현성: 동일한 흐름을 미리 만들어 놓고 재현해보며 실험 반복이 쉬움
- 전처리와 모델 단계의 파라미터를 동시에 최적화 가능
- 범주형/수치형 변수 전처리가 다를 때는 ColumnTransformer와 조합

```python
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('전처리단계1', 변환기1),
    ('전처리단계2', 변환기2),
    ('모델', 모델)
])
```
<p align="center">
  <img src="https://bait509-ubc.github.io/BAIT509/_images/column-transformer.png" align='center' width=700>
  <br>
  <sub><a href="https://bait509-ubc.github.io/BAIT509/lectures/lecture5.html#columntransformer">ColumnTransformer - Source: BAIT509-UBC </a></sub>
</p>

<p align="center">
  <img src="https://raw.githubusercontent.com/isjo-dit/Lecture/9db9fe1e3de5312d1cedb69786dd000e8b23664c/image/pipeline.png" align='center' width=600>
</p>


In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# 1) 데이터 불러오기
df = sns.load_dataset("titanic").dropna(subset=["age", "sex", "survived"])
X = df[["age", "sex"]]
y = df["survived"]

# 2) 수치형/범주형 구분
num_features = ["age"]
cat_features = ["sex"]

# 3) 전처리기 정의
numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),  # 결측치 처리
    ("scaler", StandardScaler())                    # 표준화
])
categorical_tf = Pipeline([
    ("onehot", OneHotEncoder(drop="first"))         # 원-핫 인코딩
])

# 4) ColumnTransformer로 결합
preprocessor = ColumnTransformer([
    ("num", numeric_tf, num_features),
    ("cat", categorical_tf, cat_features)
])

# 5) 전체 파이프라인
pipe = Pipeline([
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])

# 6) 학습/평가
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipe.fit(X_train, y_train)
print("테스트 정확도:", pipe.score(X_test, y_test))

from sklearn import set_config
set_config(display='diagram')
pipe

테스트 정확도: 0.7342657342657343


,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


---

## 종합 예제


In [ ]:
# 완전한 머신러닝 파이프라인
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report

np.random.seed(42)
df = sns.load_dataset('titanic')

print(df.isna().sum())
print(df.info())

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    cate

In [ ]:
# 특성과 타겟 분리
X = df[["pclass","sex","age","sibsp","parch","fare","embarked"]]
y = df['survived']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 전처리 파이프라인 정의
numeric_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns
print(numeric_features)
print(categorical_features)

# 수치형 전처리
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 전처리
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False))
])

# 전처리 결합
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 전체 파이프라인 (전처리 + 모델)
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 하이퍼파라미터 그리드
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [3, 5, None],
    'classifier__min_samples_split': [2, 5]
}

# GridSearchCV 적용
grid_search = GridSearchCV(
    full_pipeline,
    param_grid,
    cv=3,
    scoring='f1'
)
print(X_train.columns.tolist())
# 학습 및 최적화
grid_search.fit(X_train, y_train)

# 결과 출력
print(f"최적 매개변수: {grid_search.best_params_}")
print(f"교차검증 최고 점수: {grid_search.best_score_:.3f}")

# 테스트 데이터 평가
y_pred = grid_search.predict(X_test)
test_score = grid_search.score(X_test, y_test)
print(f"테스트 정확도: {test_score:.3f}")

print("\n상세 분류 보고서:")
print(classification_report(y_test, y_pred))

Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object')
Index(['sex', 'embarked'], dtype='object')
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
최적 매개변수: {'classifier__max_depth': 5, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50}
교차검증 최고 점수: 0.745
테스트 정확도: 0.759

상세 분류 보고서:
              precision    recall  f1-score   support

           0       0.81      0.90      0.85       105
           1       0.83      0.70      0.76        74

    accuracy                           0.82       179
   macro avg       0.82      0.80      0.80       179
weighted avg       0.82      0.82      0.81       179

